In [1]:
import pandas as pd

In [2]:
corpus_file = '../../data/democracy_all_predictions_130724.csv'
keyword_file = '../../data/democracy_keywordextraction_aggregated_counts_240724.csv'
df = pd.read_csv(corpus_file)
df_keyword = pd.read_csv(keyword_file)
#print(df)

/var/folders/pj/wj7m849s68qcg6_lr0j03jgh0000gn/T/ipykernel_42440/4090809804.py:3: DtypeWarning: Columns (4,5,6,8,9,10,12,13,14,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(corpus_file)


In [3]:
print(df.shape[0])
print(df_keyword.shape[0])

459931
5084


In [4]:
print(df.sample(1))

                                                 sentence country  year  \
157513  The reason is not new in Russia, which can onl...  russia  2018   

       source dimension1_r1 dimension2_r1 backsliding_r1  start_idea_r1  \
157513    bti           NaN           NaN            NaN            NaN   

       dimension0_r2 dimension1_r2  ... start_idea_r2  start_idea_r3  \
157513           NaN           NaN  ...           NaN            NaN   

       dimension0_r1 correct_dimension expalained_ambiquous  dimension0  \
157513           NaN               NaN                  NaN         NaN   

        correct_backsliding predicted_dim_roberta  \
157513                  NaN             ambiguous   

       predicted_dim_roberta_no_ambiguous predicted_backsliding_123  
157513                            liberal                       3.0  

[1 rows x 26 columns]


In [5]:
# Agregate dimensions as contry, year source dimension
df_agg_roberta = df.groupby(['country', 'year', 'source', 'predicted_dim_roberta']).size().reset_index(name='dim_roberta_agg_count')
df_agg_roberta_no_ambiguous = df.groupby(['country', 'year', 'source', 'predicted_dim_roberta_no_ambiguous']).size().reset_index(name='dim_roberta_no_ambiguous_agg_count')
# Sentiment groupings
df_agg_roberta_sentiment = df.groupby(['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta']).size().reset_index(name='backsliding_roberta_agg_count')
df_agg_roberta__no_ambiguous_sentiment = df.groupby(['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta_no_ambiguous']).size().reset_index(name='backsliding_roberta_no_ambiguous_agg_count')
# Corpus size by country, source, year
df_agg_corpus = df.groupby(['country', 'year', 'source']).size().reset_index(name='sentence_count')


In [6]:
df_keyword = pd.read_csv(keyword_file)
df_keyword.rename(columns={'dimension': 'dim_keyword_dictionary', 'count': 'dim_keyword_dictionary_agg_count'}, inplace=True)
#print(df_keyword)

In [7]:
#df_temp_1 = pd.merge(df_agg_roberta, df_agg_roberta_no_ambiguous, on=['country', 'year', 'source'], how='left')
df_temp_1 = pd.merge(df_agg_roberta, df_agg_roberta_no_ambiguous, left_on=['country', 'year', 'source', 'predicted_dim_roberta'], right_on=['country', 'year', 'source','predicted_dim_roberta_no_ambiguous'], how='outer')
df_temp_2 = pd.merge(df_temp_1, df_keyword, left_on=['country', 'year', 'source', 'predicted_dim_roberta'], right_on=['country', 'year', 'source','dim_keyword_dictionary'], how='outer')

#df_temp_3 = pd.merge(df_temp_2, df_agg_roberta_sentiment, on=['country', 'year', 'source', 'predicted_dim_roberta'], how='left')
#df_temp_3 = pd.merge(df_temp_2, df_keyword, on=['country', 'year', 'source'], how='outer')
df_agg_all =  pd.merge(df_temp_2, df_agg_corpus, on=['country', 'year', 'source'], how='left')

In [8]:
def fill_blanks(row):
    if(pd.isna(row['predicted_dim_roberta_no_ambiguous'])):
        row['predicted_dim_roberta_no_ambiguous'] = row['predicted_dim_roberta']
        row['dim_roberta_no_ambiguous_agg_count'] = 0
    if(pd.isna(row['dim_keyword_dictionary'])):
        row['dim_keyword_dictionary'] = row['predicted_dim_roberta']
        row['dim_keyword_dictionary_agg_count'] = 0
    return row
def fill_blanks_back(row):
    if(pd.isna(row['predicted_dim_roberta_no_ambiguous'])):
        row['predicted_dim_roberta_no_ambiguous'] = row['predicted_dim_roberta']
        row['backsliding_roberta_no_ambiguous_agg_count'] = 0
    return row   

In [9]:
df_agg_all = df_agg_all.apply(fill_blanks, axis=1)

In [10]:
df_back_1 = pd.merge(df_agg_roberta_sentiment, df_agg_roberta__no_ambiguous_sentiment, left_on=['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta'], right_on=['country', 'year', 'source', 'predicted_backsliding_123', 'predicted_dim_roberta_no_ambiguous'], how='outer')
df_back_1 = df_back_1.apply(fill_blanks_back, axis=1)
df_back_1 = df_back_1.rename(columns={'predicted_backsliding_123': 'backsliding'})
df_agg_back =  pd.merge(df_back_1, df_agg_corpus, on=['country', 'year', 'source'], how='left')

In [11]:
print(df_agg_back)

              country  year                      source  backsliding  \
0             albania  2002                       greco          1.0   
1             albania  2002                       greco          1.0   
2             albania  2002                       greco          1.0   
3             albania  2002                       greco          1.0   
4             albania  2002                       greco          1.0   
...               ...   ...                         ...          ...   
20880  united-kingdom  2020  freedomhouse_freedom-world          1.0   
20881  united-kingdom  2021  freedomhouse_freedom-world          1.0   
20882  united-kingdom  2022  freedomhouse_freedom-world          1.0   
20883  united-kingdom  2022  freedomhouse_freedom-world          3.0   
20884  united-kingdom  2023  freedomhouse_freedom-world          1.0   

      predicted_dim_roberta  backsliding_roberta_agg_count  \
0                 ambiguous                           52.0   
1          

In [12]:
from datetime import datetime
file_name = datetime.now().strftime('../../data/democracy_aggregate_counts_for_backsliding_%d%m%y')
print(f"Saving {file_name}.csv")
df_agg_back.to_csv(f"{file_name}.csv", index=False)
print(f"Saving {file_name}.xlsx")
df_agg_back.to_excel(f"{file_name}.xlsx", index=False)

Saving ../../data/democracy_aggregate_counts_for_backsliding_250724.csv
Saving ../../data/democracy_aggregate_counts_for_backsliding_250724.xlsx


In [13]:
from datetime import datetime
file_name = datetime.now().strftime('../../data/democracy_aggregate_counts_for_dimenions_%d%m%y')
print(f"Saving {file_name}.csv")
df_agg_all.to_csv(f"{file_name}.csv", index=False)
print(f"Saving {file_name}.xlsx")
df_agg_all.to_excel(f"{file_name}.xlsx", index=False)

Saving ../../data/democracy_aggregate_counts_for_dimenions_250724.csv
Saving ../../data/democracy_aggregate_counts_for_dimenions_250724.xlsx


In [14]:
print("done")

done


In [15]:
test_df = pd.read_csv(f"{file_name}.csv")
print(test_df.columns)

Index(['country', 'year', 'source', 'predicted_dim_roberta',
       'dim_roberta_agg_count', 'predicted_dim_roberta_no_ambiguous',
       'dim_roberta_no_ambiguous_agg_count', 'dim_keyword_dictionary',
       'dim_keyword_dictionary_agg_count', 'sentence_count'],
      dtype='object')
